In [5]:
import json
import os

path = './json_searchs'
json_searchs = {}
for file in os.listdir(path):
    with open(f"{path}/{file}", "r", encoding="utf-8") as f:
        json_searchs[file] = json.loads(f.read())["images_results"]
    
    print(f"Loaded {file}: {len(json_searchs[file])} images")


Loaded vestido decote coracao.json: 100 images
Loaded vestido decote frente unica.json: 100 images
Loaded vestido decote ombro a ombro.json: 100 images
Loaded vestido decote redondo.json: 100 images
Loaded vestido decote reto.json: 99 images
Loaded vestido decote tomara que caia.json: 100 images
Loaded vestido decote um ombro so.json: 100 images
Loaded vestido decote v.json: 100 images


In [6]:
import boto3
import requests
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
from tqdm import tqdm

# Função para carregar imagem
def load_image(url_or_path):
    try:
        if url_or_path.startswith("http://") or url_or_path.startswith("https://"):

            # Fazer o download da imagem
            response = requests.get(url_or_path)
            response.raise_for_status()  # Verifica se houve erro na requisição

            # Carregar a imagem no Pillow
            image = Image.open(BytesIO(response.content))

            return image
        else:
            return Image.open(url_or_path)
    except Exception as e:
        print(f"Erro ao carregar a imagem: {url_or_path}. Detalhes: {e}")
        return None     

def img_augmentation(image, rotation=False, flip_horizontal=False, flip_vertical=False, black_and_white=False):
    transformations = {
        "original": image,
    }
    
    # Rotação
    if rotation:
        for angle in [90, 180, 270]:
            transformations[f"rotation_{angle}"] = image.rotate(angle)

    # Flip horizontal
    if flip_horizontal:
        transformations["flip_horizontal"] = image.transpose(Image.FLIP_LEFT_RIGHT)

    # Flip vertical
    if flip_vertical:
        transformations["flip_vertical"] = image.transpose(Image.FLIP_TOP_BOTTOM)
    
    # Preto e branco
    if black_and_white:
        transformations["black_and_white"] = image.convert("L")
    
    return transformations

In [7]:
limit_files = 50

df = []

for search in json_searchs:
    print(f"Search: {search}")
    folder_name = "./img/" + search.split(".")[0]
    for i, image in enumerate(json_searchs[search]):
        if i >= limit_files:
            break

        if os.path.exists(f"{folder_name}/{i}-original.jpg"):
            # obtem as imagens
            for transformation in ["original", "rotation_90", "rotation_180", "rotation_270", "flip_horizontal", "flip_vertical", "black_and_white"]:
                path = os.path.abspath(f"{folder_name}/{i}-{transformation}.jpg")

                img_row = {
                    "path": path,
                    "search": search.split(".")[0],
                    "image": i,
                    "id_produto": f"{search.split('.')[0]}-{i}-{transformation}",
                    "transformation": transformation,
                    "link": image["original"],
                    "width": Image.open(path).width,
                    "height": Image.open(path).height
                }
                df.append(img_row)
            continue
        
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)

        link = image["original"]

        # Carregar a imagem
        img = load_image(link)
        if img is None:
            continue # Pular para a próxima imagem

        # Salvar a imagem
        try:
            for transformation, img_transformed in img_augmentation(
                    img,
                    rotation=True,
                    flip_horizontal=True,
                    flip_vertical=True,
                    black_and_white=True
                ).items():
                img_transformed.save(f"{folder_name}/{i}-{transformation}.jpg")

                path = os.path.abspath(f"{folder_name}/{i}-{transformation}.jpg")

                img_row = {
                    "path": path,
                    "search": search.split(".")[0],
                    "image": i,
                    "id_produto": f"{search.split('.')[0]}-{i}-{transformation}",
                    "transformation": transformation,
                    "link": link,
                    "width": img_transformed.width,
                    "height": img_transformed.height
                }
                df.append(img_row)

            # print(f"Image {i}:", image["original"])
        except Exception as e:
            # print(f"Erro ao salvar a imagem: {image['original']}. Detalhes: {e}")
            pass
    
df = pd.DataFrame(df)

Search: vestido decote coracao.json
Erro ao carregar a imagem: https://msstoreoficial.com.br/wp-content/uploads/2020/08/vestido_corselet-400x498.jpg. Detalhes: 406 Client Error: Not Acceptable for url: https://msstoreoficial.com.br/wp-content/uploads/2020/08/vestido_corselet-400x498.jpg
Erro ao carregar a imagem: https://www.simmetriamodafesta.com.br/crisoft/fotos/221233a32criacaodesitescrisoft.jpg. Detalhes: 406 Client Error: Not Acceptable for url: https://www.simmetriamodafesta.com.br/crisoft/fotos/221233a32criacaodesitescrisoft.jpg
Erro ao carregar a imagem: https://lookaside.fbsbx.com/lookaside/crawler/media/?media_id=2156240637822576. Detalhes: cannot identify image file <_io.BytesIO object at 0x0000022CFBEF2B60>
Erro ao carregar a imagem: https://cdn0.casamentos.com.br/article-dress/5432/original/1280/jpg/m702345.jpeg. Detalhes: 403 Client Error: Forbidden for url: https://cdn0.casamentos.com.br/article-dress/5432/original/1280/jpg/m702345.jpeg
Search: vestido decote frente unic

In [8]:
print(df.shape)
df.head()

(2513, 8)


,path,search,image,id_produto,transformation,link,width,height
0,c:\Users\Nicole.Souza.HOMLABNOT002581\Desktop\...,vestido decote coracao,0,vestido decote coracao-0-original,original,http://acdn.mitiendanube.com/stores/001/552/35...,640,1138
1,c:\Users\Nicole.Souza.HOMLABNOT002581\Desktop\...,vestido decote coracao,0,vestido decote coracao-0-rotation_90,rotation_90,http://acdn.mitiendanube.com/stores/001/552/35...,640,1138
2,c:\Users\Nicole.Souza.HOMLABNOT002581\Desktop\...,vestido decote coracao,0,vestido decote coracao-0-rotation_180,rotation_180,http://acdn.mitiendanube.com/stores/001/552/35...,640,1138
3,c:\Users\Nicole.Souza.HOMLABNOT002581\Desktop\...,vestido decote coracao,0,vestido decote coracao-0-rotation_270,rotation_270,http://acdn.mitiendanube.com/stores/001/552/35...,640,1138
4,c:\Users\Nicole.Souza.HOMLABNOT002581\Desktop\...,vestido decote coracao,0,vestido decote coracao-0-flip_horizontal,flip_horizontal,http://acdn.mitiendanube.com/stores/001/552/35...,640,1138


In [9]:
df["search"].value_counts()

search
vestido decote tomara que caia    329
vestido decote frente unica       322
vestido decote reto               322
vestido decote um ombro so        322
vestido decote v                  322
vestido decote coracao            308
vestido decote ombro a ombro      294
vestido decote redondo            294
Name: count, dtype: int64

In [10]:
from sentence_transformers import SentenceTransformer, util

# We use the original clip-ViT-B-32 for encoding images
img_model = SentenceTransformer('clip-ViT-B-32')

C:\Users\Nicole.Souza.HOMLABNOT002581\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:

from typing import List

def get_img_embeddings_in_batches(
    df: pd.DataFrame,
    path_column: str,
    img_model: SentenceTransformer,
    batch_size: int = 16
) -> pd.DataFrame:
    """
    Função que processa imagens em batches para evitar sobrecarga de memória.

    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os caminhos das imagens.
        path_column (str): Nome da coluna no DataFrame com os caminhos das imagens.
        img_model (SentenceTransformer): Modelo para geração de embeddings das imagens.
        batch_size (int): Tamanho do batch para processar as imagens.

    Retorna:
        pd.DataFrame: DataFrame com os embeddings das imagens.
    """
    emb_columns = [f"emb_img_{i}" for i in range(512)]
    emb_dict = {}

    file_names = df[path_column].values
    ids = df["id_produto"].values

    # Processar as imagens em batches
    from tqdm import tqdm

    for start_idx in tqdm(range(0, len(file_names), batch_size), desc="Processando batches", position=0, leave=True):
        end_idx = min(start_idx + batch_size, len(file_names))
        batch_files = file_names[start_idx:end_idx]
        batch_ids = ids[start_idx:end_idx]

        valid_images = []
        valid_ids = []

        # tqdm.write("Carregando imagens...")  # Substituir print
        for img_path, img_id in tqdm(
            zip(batch_files, batch_ids),
            total=len(batch_files),
            desc="Carregando imagens",
            position=1,
            leave=False,
        ):
            img = load_image(img_path)
            if img is not None:
                valid_images.append(img)
                valid_ids.append(img_id)

        if valid_images:
            # tqdm.write("Gerando embeddings...")  # Substituir print
            batch_embeddings = img_model.encode(valid_images, show_progress_bar=False)

            for img_id, emb in zip(valid_ids, batch_embeddings):
                emb_dict[img_id] = emb

    # Criar DataFrame com embeddings
    df_embs = pd.DataFrame.from_dict(emb_dict, orient='index')
    df_embs.columns = emb_columns
    df_embs["id_produto"] = df_embs.index

    # Inserir NaN para produtos que falharam
    missing_ids = df[~df["id_produto"].isin(df_embs["id_produto"])]
    for missing_id in missing_ids["id_produto"]:
        df_embs = pd.concat([
            df_embs,
            pd.DataFrame([[np.nan] * 512 + [missing_id]], columns=emb_columns + ["id_produto"])
        ])

    # Resetar índice
    df_embs = df_embs.reset_index(drop=True)

    return df_embs


In [12]:
df_embs = get_img_embeddings_in_batches(df, "path", img_model, batch_size=16)

Processando batches: 100%|██████████| 158/158 [10:02<00:00,  3.81s/it]


In [13]:
df = df.merge(df_embs, on="id_produto")

In [14]:
df.to_parquet("./img_embeddings_teste_pesquisa.parquet", index=False)

In [12]:
# subespaços
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from umap.umap_ import UMAP

# Função que aplica LDA: recebe um df, as colunas desse df que contêm os embeddings e o número de componentes que se deseja obter
def apply_lda(df, colunas, n_components=2, lda=None, column_target='COR_PREDOMINANTE'):
    """
    Aplica LDA nos embeddings para redução de dimensionalidade com base na coluna passada como target,
    ignorando valores nulos na coluna de target e retornando um novo DataFrame com as colunas desejadas.
    """
    df_filtered = df.copy()

    if not lda:
        # Inicializando o modelo LDA
        lda = LinearDiscriminantAnalysis(n_components=n_components)

        # Filtrando os dados para remover nulos na coluna 'COR_PREDOMINANTE'
        df_filtered = df_filtered.dropna(subset=[f'{column_target}'])

        # Treinando o modelo e transformando os dados
        componentes = lda.fit_transform(df_filtered[colunas], df_filtered[f'{column_target}'])

        # Criando um novo DataFrame com as colunas desejadas
        df_lda = df_filtered[['id_produto', f'{column_target}']].copy()

    else:
        componentes = lda.transform(df_filtered[colunas])

        # Criando um novo DataFrame com as colunas desejadas
        df_lda = df_filtered[['id_produto']].copy()

    for i in range(n_components):
        df_lda[f'EMB_LDA_{i}'] = componentes[:, i]

    return df_lda, lda

def apply_umap(df, colunas, n_components=2, umap=None):
    """
    Aplica UMAP nos embeddings para redução de dimensionalidade
    """

    if umap is None:
        # Removendo linhas com valores nulos para treinar o modelo

        # Inicializando o modelo t-SNE
        umap = UMAP(n_components=n_components, random_state=42)

        # Treinando o modelo e transformando os dados
        componentes = umap.fit_transform(df[colunas])

        # Criando um novo DataFrame com as colunas desejadas
        df_umap = df[['id_produto']].copy() 
    else:
        # Transformando os dados
        componentes = umap.transform(df[colunas])

        # Criando um novo DataFrame com as colunas desejadas
        df_umap = df[['id_produto']].copy()

    for i in range(n_components):
        df_umap[f'EMB_UMAP_{i}'] = componentes[:, i]

    return df_umap, umap

def calcular_centroides(df, colunas, target_column):
    """
    Calcula os centróides para cada categoria em 'target_column' com base nas colunas fornecidas.
    Retorna um DataFrame com os centróides e a categoria correspondente.
    """
    
    # 1. Calcular os centróides para cada {target_column}
    df_centroides = df.drop(columns={'id_produto'}).groupby(f'{target_column}').mean().reset_index()

    # 2. Adicionar a coluna 'id_produto' aos centróides
    df_centroides['id_produto'] = df_centroides[f'{target_column}'].apply(lambda x: f'CLUSTER_{x}')

    # 3. Adicionar uma coluna para identificar os centróides
    df_centroides['is_centroid'] = True
    
    df['is_centroid'] = False  # Para os dados originais

    # 4. Concatenar os dados originais com os centróides
    df_combined = pd.concat([df, df_centroides], ignore_index=True)

    return df_combined

import plotly.graph_objects as go

def plot_clusters_with_centroids(
    df_combined, 
    target_column, 
    embedding_columns, 
    color_dict=None, 
    title="Visualização de Clusters com Centróides", 
    apply_umap=False, 
    umap_params=None,
    is_1d=False
):
    """
    Cria um gráfico de dispersão interativo com os clusters e seus centróides.
    
    Parâmetros:
    - df_combined: DataFrame contendo os dados originais e os centróides.
    - target_column: Nome da coluna que contém as categorias dos clusters.
    - embedding_columns: Lista com os nomes das colunas de embeddings.
    - color_dict: Dicionário com as cores específicas para cada categoria. Default: cores automáticas.
    - title: Título do gráfico. Default: "Visualização de Clusters com Centróides".
    - apply_umap: Booleano indicando se UMAP deve ser aplicado. Default: False.
    - umap_params: Dicionário com parâmetros para o UMAP. Default: None.
    - is_1d: Booleano indicando se os dados são unidimensionais. Default: False.
    
    Retorna:
    - Objeto Figure do Plotly.
    """
    # Aplicar UMAP se solicitado
    if apply_umap:
        umap_params = umap_params or {'n_neighbors': 15, 'n_components': 2, 'random_state': 42}
        umap = UMAP(**umap_params)
        
        embeddings = df_combined[embedding_columns].values
        umap_embeddings = umap.fit_transform(embeddings)

        # Adicionar os embeddings reduzidos ao DataFrame
        df_combined['UMAP_0'] = umap_embeddings[:, 0]
        if not is_1d:
            df_combined['UMAP_1'] = umap_embeddings[:, 1]

        # Ajustar as colunas para o gráfico
        embedding_columns = ['UMAP_0'] if is_1d else ['UMAP_0', 'UMAP_1']

    # Separar dados originais e centróides
    df_original = df_combined[df_combined['is_centroid'] == False].copy()
    df_original['Tipo'] = 'Original'

    df_centroids = df_combined[df_combined['is_centroid'] == True].copy()
    df_centroids['Tipo'] = 'Centróide'

    # Gerar um dicionário de cores se não fornecido
    if color_dict is None:
        categorias = df_original[target_column].unique()
        # Setando paleta de cores do tableau
        color_palette = [
            '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
            '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
            '#ff9896', '#aec7e8', '#ffbb78', '#98df8a'
        ]
        color_dict = {categoria: color_palette[i % len(color_palette)] for i, categoria in enumerate(categorias)}

    # Criação do gráfico com Plotly
    fig = go.Figure()

    # Adicionar os pontos dos dados originais por categoria
    for categoria, cor in color_dict.items():
        subset = df_original[df_original[target_column] == categoria]
        fig.add_trace(go.Scatter(
            x=subset[embedding_columns[0]],
            y=[0] * len(subset) if is_1d else subset[embedding_columns[1]],
            mode='markers',
            marker=dict(color=cor, size=8, line=dict(width=1, color='black')),
            name=f'{categoria}',
            hoverinfo='text',
            text=subset['id_produto']
        ))

    # Adicionar os centróides com cor fixa
    fig.add_trace(go.Scatter(
        x=df_centroids[embedding_columns[0]],
        y=[0] * len(df_centroids) if is_1d else df_centroids[embedding_columns[1]],
        mode='markers',
        marker=dict(color='red', symbol='x', size=10, line=dict(width=0.5)),
        name='Centróides',
        hoverinfo='text',
        text=df_centroids['id_produto']
    ))

    # Configuração do layout
    fig.update_layout(
        title=title,
        xaxis_title=f'{embedding_columns[0]}',
        yaxis_title='' if is_1d else f'{embedding_columns[1]}',
        legend_title='Legenda',
        template='plotly_white',
        hovermode='closest'
    )

    if is_1d:
        fig.update_layout(
            yaxis=dict(showgrid=False, showticklabels=False, zeroline=False)
        )

    return fig

In [13]:
emb_cols = [col for col in df.columns if col.startswith("emb_img_")]
df_lda_decote, lda_decote = apply_lda(df, emb_cols, n_components=5, lda=None, column_target='search')

colunas_lda = [col for col in df_lda_decote.columns if col.startswith('EMB_LDA')]

# Calcular os centróides de df_lda_modelagem_saia
df_combined_decote = calcular_centroides(df_lda_decote, colunas_lda, 'search')

In [14]:
fig = plot_clusters_with_centroids(
    df_combined=df_combined_decote,
    target_column='search',
    embedding_columns=colunas_lda,
    color_dict=None,
    title="Visualização com UMAP",
    apply_umap=True,
    is_1d=False
)
fig.show()

C:\Users\Nicole.Souza.HOMLABNOT002581\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [15]:
path = 'C:/Users/Nicole.Souza.HOMLABNOT002581/Desktop/RECONHECIMENTO DE PRODUTOS SIMILARES/StyleMatchAI/APLICACAO/modules/datasets'
if os.path.exists(path+'/img_embeddings.parquet'):
    df_img_embs = pd.read_parquet(path+'/img_embeddings.parquet')
    print("Arquivo de embeddings de imagens encontrado!")

path = 'C:/Users/Nicole.Souza.HOMLABNOT002581/Desktop/RECONHECIMENTO DE PRODUTOS SIMILARES/StyleMatchAI/TESTES E EXPERIMENTOS/datasets'
if os.path.exists(path+'/base_animale_categorizada_4o.parquet'):
    base_animale_categorizada = pd.read_parquet(path+'/base_animale_categorizada_4o.parquet')
    print("Arquivo de categorias encontrado!")

df_img_embs = pd.merge(df_img_embs, base_animale_categorizada[['id_produto', 'grupo_produto', 'DECOTE_OU_GOLA']], on='id_produto', how='left')


Arquivo de embeddings de imagens encontrado!
Arquivo de categorias encontrado!


In [16]:
df_img_embs = df_img_embs[df_img_embs["grupo_produto"] == 'vestido']
df_img_embs['DECOTE_OU_GOLA'].value_counts()

DECOTE_OU_GOLA
V                  74
FRENTE_UNICA       30
RETO               29
REDONDO            22
UM_OMBRO_SO        15
CORACAO            14
TOMARA_QUE_CAIA    14
OMBRO_A_OMBRO       9
GOLA_ALTA           3
N/A                 2
DEGAGE              2
GOLA_POLO           1
Name: count, dtype: int64

In [17]:
import torch


def classify(df, colunas_lda, centroides):
    """
    Classifica os dados de acordo com os centróides fornecidos.
    Retorna um DataFrame com a classificação.
    """
    img_embeddings_lda = df[colunas_lda].values
    
    centroides_nomes = centroides['id_produto'].values
    centroides_values = centroides[colunas_lda].values

    centroides_values = torch.tensor(centroides_values, dtype=torch.float32)

    img_embeddings_lda = torch.tensor(img_embeddings_lda, dtype=torch.float32)

    euclidean_dist = torch.cdist(img_embeddings_lda, centroides_values, p=2)

    # Encontrar o índice do centróide mais próximo
    idx_min = torch.argmin(euclidean_dist, dim=1)   

    # Adicionar a classificação ao DataFrame
    df['class'] = centroides_nomes[idx_min]

    return df



In [18]:
centroides = df_combined_decote[df_combined_decote['is_centroid'] == True]

In [19]:
emb_cols = [col for col in df_img_embs.columns if col.startswith("emb_img_")]

df_img_lda_decote, lda_decote = apply_lda(df_img_embs, emb_cols, n_components=5, lda=lda_decote)

colunas_lda = [col for col in df_img_lda_decote.columns if col.startswith('EMB_LDA')]

df_img_lda_decote = classify(df_img_lda_decote, colunas_lda, centroides)

In [20]:
df_img_embs = pd.merge(df_img_embs, df_img_lda_decote[['id_produto', 'class']], on='id_produto', how='left')

In [21]:
df_img_embs[["DECOTE_OU_GOLA", "class"]].value_counts()

DECOTE_OU_GOLA   class                                 
V                CLUSTER_vestido decote coracao            26
                 CLUSTER_vestido decote frente unica       24
FRENTE_UNICA     CLUSTER_vestido decote frente unica       15
RETO             CLUSTER_vestido decote frente unica       10
                 CLUSTER_vestido decote reto                8
REDONDO          CLUSTER_vestido decote frente unica        7
RETO             CLUSTER_vestido decote coracao             7
V                CLUSTER_vestido decote redondo             7
CORACAO          CLUSTER_vestido decote frente unica        6
UM_OMBRO_SO      CLUSTER_vestido decote coracao             6
FRENTE_UNICA     CLUSTER_vestido decote coracao             6
V                CLUSTER_vestido decote reto                6
FRENTE_UNICA     CLUSTER_vestido decote reto                5
UM_OMBRO_SO      CLUSTER_vestido decote frente unica        5
TOMARA_QUE_CAIA  CLUSTER_vestido decote frente unica        5
OMBRO_A_OMBRO 

In [22]:
df_img_embs[["class", "DECOTE_OU_GOLA"]].value_counts()

class                                   DECOTE_OU_GOLA 
CLUSTER_vestido decote coracao          V                  26
CLUSTER_vestido decote frente unica     V                  24
                                        FRENTE_UNICA       15
                                        RETO               10
CLUSTER_vestido decote reto             RETO                8
CLUSTER_vestido decote frente unica     REDONDO             7
CLUSTER_vestido decote coracao          RETO                7
CLUSTER_vestido decote redondo          V                   7
CLUSTER_vestido decote reto             V                   6
CLUSTER_vestido decote frente unica     CORACAO             6
CLUSTER_vestido decote coracao          UM_OMBRO_SO         6
                                        FRENTE_UNICA        6
CLUSTER_vestido decote um ombro so      V                   5
CLUSTER_vestido decote coracao          OMBRO_A_OMBRO       5
CLUSTER_vestido decote reto             FRENTE_UNICA        5
CLUSTER_vestid

---

In [15]:
import pandas as pd

df = pd.read_parquet("./img_embeddings_teste_pesquisa.parquet")
df.shape

(2513, 520)

In [18]:
emb_cols = [col for col in df.columns if col.startswith("emb_img_")]
target = "search"

In [19]:
# Criando um modelo de classificação com uma MLP simples
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Separar os dados em treino e teste
X = df[emb_cols].values
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar o modelo
clf = MLPClassifier(hidden_layer_sizes=(512, 256), max_iter=1000, random_state=42)

# Treinar o modelo
clf.fit(X_train, y_train)

# Fazer previsões
y_pred = clf.predict(X_test)

# Avaliar o modelo
print(classification_report(y_test, y_pred))

                                precision    recall  f1-score   support

        vestido decote coracao       0.79      0.83      0.81        58
   vestido decote frente unica       0.97      0.96      0.96        70
  vestido decote ombro a ombro       0.80      0.89      0.84        54
        vestido decote redondo       0.97      0.95      0.96        60
           vestido decote reto       0.97      0.95      0.96        65
vestido decote tomara que caia       0.84      0.84      0.84        64
    vestido decote um ombro so       0.97      0.88      0.92        72
              vestido decote v       0.97      0.98      0.98        60

                      accuracy                           0.91       503
                     macro avg       0.91      0.91      0.91       503
                  weighted avg       0.91      0.91      0.91       503



In [20]:
df_img_embs["classe predicao"] = clf.predict(df_img_embs[emb_cols].values)

df_img_embs[["DECOTE_OU_GOLA", "classe predicao"]].value_counts()

DECOTE_OU_GOLA   classe predicao               
V                vestido decote v                  30
RETO             vestido decote reto               17
FRENTE_UNICA     vestido decote reto               14
V                vestido decote reto               14
                 vestido decote redondo            12
TOMARA_QUE_CAIA  vestido decote reto               11
CORACAO          vestido decote reto                9
REDONDO          vestido decote reto                8
V                vestido decote coracao             7
RETO             vestido decote coracao             7
OMBRO_A_OMBRO    vestido decote v                   5
V                vestido decote frente unica        5
REDONDO          vestido decote v                   5
UM_OMBRO_SO      vestido decote v                   5
FRENTE_UNICA     vestido decote frente unica        5
V                vestido decote ombro a ombro       4
OMBRO_A_OMBRO    vestido decote redondo             3
RETO             vestido decote v 

: 